In [52]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
#popular optimization strategy that uses gradient descent 
from tensorflow.keras.optimizers import Adam
#to save our model periodically as checkpoints for loading later
from tensorflow.keras.callbacks import ModelCheckpoint
#what types of layers do we want our model to have?
from tensorflow.keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from utils import INPUT_SHAPE, batch_generator

In [53]:
np.random.seed(0) #for debugging, allows for reproducible (deterministic) results 

In [54]:
df = pd.read_csv('../data/driving_log.csv', names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed'])

In [55]:
df.head()

,center,left,right,steering,throttle,reverse,speed
0,/Users/anthonysmacbook/DSI/week_7/How_to_simul...,/Users/anthonysmacbook/DSI/week_7/How_to_simul...,/Users/anthonysmacbook/DSI/week_7/How_to_simul...,0.0,0.0,0,0.000007
1,/Users/anthonysmacbook/DSI/week_7/How_to_simul...,/Users/anthonysmacbook/DSI/week_7/How_to_simul...,/Users/anthonysmacbook/DSI/week_7/How_to_simul...,0.0,0.0,0,0.000002
2,/Users/anthonysmacbook/DSI/week_7/How_to_simul...,/Users/anthonysmacbook/DSI/week_7/How_to_simul...,/Users/anthonysmacbook/DSI/week_7/How_to_simul...,0.0,0.0,0,0.000008
3,/Users/anthonysmacbook/DSI/week_7/How_to_simul...,/Users/anthonysmacbook/DSI/week_7/How_to_simul...,/Users/anthonysmacbook/DSI/week_7/How_to_simul...,0.0,0.0,0,0.000009
4,/Users/anthonysmacbook/DSI/week_7/How_to_simul...,/Users/anthonysmacbook/DSI/week_7/How_to_simul...,/Users/anthonysmacbook/DSI/week_7/How_to_simul...,0.0,0.0,0,0.000013


In [56]:
df.center = df.center.str.replace('/Users/anthonysmacbook/DSI/week_7/How_to_simulate_a_self_driving_car', '..')
df.left = df.left.str.replace('/Users/anthonysmacbook/DSI/week_7/How_to_simulate_a_self_driving_car', '..')
df.right = df.right.str.replace('/Users/anthonysmacbook/DSI/week_7/How_to_simulate_a_self_driving_car', '..')

In [57]:
X = df[['center', 'left', 'right']].values
y = df['steering'].values

In [58]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

In [59]:
model = Sequential()
model.add(Lambda(lambda x: x/127.5-1.0, input_shape=INPUT_SHAPE))
model.add(Conv2D(filters=24, kernel_size=(5, 5), activation='elu', strides=2))
model.add(Conv2D(filters=36, kernel_size=(5, 5), activation='elu', strides=2))
model.add(Conv2D(filters=48, kernel_size=(5, 5), activation='elu', strides=2))
#     model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2, 2)))
#     model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2, 2)))
#     model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='elu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='elu'))
# model.add(Dropout(args.keep_prob))
model.add(Dropout(rate=0.5))
model.add(Flatten())
model.add(Dense(100, activation='elu'))
model.add(Dense(50, activation='elu'))
model.add(Dense(10, activation='elu'))
model.add(Dense(1))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_6 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 78, 158, 24)       1824      
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 37, 77, 36)        21636     
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 17, 37, 48)        43248     
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 15, 35, 64)        27712     
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 13, 33, 64)        36928     
_________________________________________________________________
dropout_4 (Dropout)          (None, 13, 33, 64)       

In [60]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.h5',
                                 monitor='val_loss',
                                 verbose=0,
                                 save_best_only=True,
                                 mode='auto')

In [61]:
  model.compile(loss='mean_squared_error', optimizer=Adam(lr=1.0e-4))

In [ ]:
model.fit(x=batch_generator('../data', X_train, y_train, 40, True),
                        steps_per_epoch=20000,
                        epochs=3,
                        max_queue_size=1,
                        validation_data=batch_generator('../data', X_valid, y_valid, 40, False),
                        validation_steps=len(X_valid),
                        callbacks=[checkpoint],
                        verbose=1)

Epoch 1/3
  315/20000 [..............................] - ETA: 59:33 - loss: 0.0370